In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Leer el archivo Excel
df = pd.read_excel('GPT - Circular Line Chart.xlsx')

# Asegurar que las columnas de fechas estén en formato datetime
date_columns = ['Project Baseline Finish', 'M-1 Finish', 'M-2 Finish', 'M-3 Finish', 'Current Finish']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

# Ordenar el DataFrame por 'Project Baseline Finish' en orden ascendente
df = df.sort_values('Project Baseline Finish').reset_index(drop=True)

# Obtener la primera parte de 'Activity Name' para las etiquetas después de ordenar
activity_labels = df['Activity Name'].str.split(' - ', n=1).str[0].tolist()

# Definir la fecha de referencia (1 de enero de 2023)
start_date = pd.Timestamp('2023-01-01')

# Normalizar las fechas a una escala radial
def normalize_dates(dates, start_date):
    """Convierte fechas en valores radiales donde el 1 de enero de 2023 es radio 0."""
    return [(date - start_date).days / 365.25 for date in dates]

# Normalizar cada conjunto de fechas
baseline_radial = normalize_dates(df['Project Baseline Finish'], start_date)
current_radial = normalize_dates(df['Current Finish'], start_date)
m1_radial = normalize_dates(df['M-1 Finish'], start_date)
m2_radial = normalize_dates(df['M-2 Finish'], start_date)
m3_radial = normalize_dates(df['M-3 Finish'], start_date)


# Cerrar el círculo añadiendo el primer valor al final
baseline_radial.append(baseline_radial[0])
current_radial.append(current_radial[0])
m1_radial.append(m1_radial[0])
m2_radial.append(m2_radial[0])
m3_radial.append(m3_radial[0])


# Calcular ángulos para los puntos en el círculo
num_points = len(df)
angles = [n / num_points * 2 * np.pi for n in range(num_points)]
angles += angles[:1]  # Cerrar el círculo

# Configurar la gráfica
plt.figure(figsize=(12, 12), dpi=100)
ax = plt.subplot(111, polar=True)
ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)

# Añadir anillos para cada año
for year in range(2023, 2030):  # Ajusta el rango según tus datos
    ring_radius = year - 2023  # Ajuste para que el año 2024 sea radio 1
    ax.plot(np.linspace(0, 2 * np.pi, 100), [ring_radius] * 100, linestyle='--', color='grey', linewidth=1, alpha=0.6)
    if ring_radius >= 0:
        ax.text(0, ring_radius, str(year), ha='center', va='bottom', fontsize=10, color='black')

# Añadir la circunferencia para el día de hoy
today = pd.Timestamp('today').normalize()
today_radial = normalize_dates([today], start_date)[0]
theta = np.linspace(0, 2 * np.pi, 500)
ax.plot(theta, [today_radial]*len(theta), linestyle='--', color='red', linewidth=1, label='Today')

# Añadir etiqueta con la fecha de hoy en la parte inferior del círculo, en el segundo radio
formatted_today = today.strftime('%d-%b-%Y')  # Formato 'DD-MMM-YYYY', e.g., '04-Dec-2023'
ax.text(np.pi, 2, formatted_today, ha='center', va='top', fontsize=10, color='red')

# Añadir radios como líneas punteadas
max_radius = max(max(baseline_radial), max(m1_radial), max(m2_radial), max(m3_radial), max(current_radial), today_radial)
for angle in angles[:-1]:  # Excluimos el último ángulo para evitar duplicado
    ax.plot([angle, angle], [0, max_radius], linestyle=':', color='grey', linewidth=1, alpha=0.6)

# Graficar cada conjunto de datos con 'zorder'
ax.plot(angles, baseline_radial, linestyle='-', marker='o', color='blue', label='Project Baseline Finish', zorder=3)
ax.plot(angles, current_radial, linestyle='-', marker='o', color='green', label='Current Finish', zorder=2)
ax.plot(angles, m1_radial, linestyle='-', marker='o', color='orange', label='M-1 Finish', zorder=2)
ax.plot(angles, m2_radial, linestyle='-', marker='o', color='red', label='M-2 Finish', zorder=2)
ax.plot(angles, m3_radial, linestyle='-', marker='o', color='purple', label='M-3 Finish', zorder=2)


# Añadir etiquetas de actividades
ax.set_xticks(angles[:-1])
ax.set_xticklabels(activity_labels, fontsize=12)

# Mejorar la estética del gráfico
ax.spines['polar'].set_visible(False)
ax.yaxis.set_visible(False)

# Añadir leyenda
plt.legend(loc='lower right', fontsize=10)

# Añadir título
plt.title('Circular Line Chart: Project Baseline vs Monthly Finishes', va='bottom')

# Guardar el gráfico como PNG sin fondo
plt.savefig('Circular_Line_Chart.png', format='png', transparent=True)

# Mostrar el gráfico
plt.show()
